In [ ]:
#rtWIZARD Dry Run Protocol v1.0.0
#Will dry run protocol v1.0.0 using the same tip

#import opentrons.simulate if you want to simulate the protocol first
import opentrons.simulate
requirements = {"apiLevel": "2.13"}
protocol = opentrons.simulate.get_protocol_api('2.13')
protocol.home()    #homes motors

#import opentrons.execute if you want to execute the protocol on the OT-2
#import opentrons.execute
#protocol = opentrons.execute.get_protocol_api("2.13")
#protocol.home()    #homes motors

#assigns each OT2 slot a custom labware definition that corresponds to the cutout
#BEFORE EXECUTING: Run labware position check and copy/paste the Jupyter Notebook labware offsets as shown below:
'''
slot1 = protocol.load_labware("rtwizard_slots_147", 1)
slot1.set_offset(x=1.30, y=-3.50, z=-4.00)
'''
#paste offsets below:

labware_1 = protocol.load_labware("rtwizard_slot_147_rapid_asfv_ag", location="1")
labware_1.set_offset(x=-6.50, y=-4.30, z=-5.00)

labware_2 = protocol.load_labware("rtwizard_slot_258_rapid_asfv_ag", location="2")
labware_2.set_offset(x=-1.20, y=-4.40, z=-5.00)

labware_3 = protocol.load_labware("rtwizard_slot_369_rapid_asfv_ag", location="3")
labware_3.set_offset(x=4.00, y=-4.00, z=-5.00)

labware_4 = protocol.load_labware("rtwizard_slot_147_rapid_asfv_ag", location="4")
labware_4.set_offset(x=-6.50, y=-2.00, z=-4.60)

labware_5 = protocol.load_labware("rtwizard_slot_258_rapid_asfv_ag", location="5")
labware_5.set_offset(x=-1.00, y=-2.00, z=-4.00)

labware_6 = protocol.load_labware("rtwizard_slot_369_rapid_asfv_ag", location="6")
labware_6.set_offset(x=3.80, y=-2.00, z=-4.00)

labware_7 = protocol.load_labware("rtwizard_slot_147_rapid_asfv_ag", location="7")
labware_7.set_offset(x=-6.10, y=-0.30, z=-4.00)

labware_8 = protocol.load_labware("rtwizard_slot_258_rapid_asfv_ag", location="8")
labware_8.set_offset(x=-0.40, y=0.50, z=-4.00)

labware_9 = protocol.load_labware("rtwizard_slot_369_rapid_asfv_ag", location="9")
labware_9.set_offset(x=4.90, y=0.20, z=-4.00)

labware_10 = protocol.load_labware("rtwizard_48_wellplate_sarstedt_1100ul_tubes", location="10")
labware_10.set_offset(x=-1.00, y=2.00, z=-3.00)

labware_11 = protocol.load_labware("opentrons_96_tiprack_300ul", location="11")
labware_11.set_offset(x=-0.60, y=7.40, z=0.00)

#standardize names
sample_plate = labware_10
tips = labware_11

#comment or uncomment depending on pipette setup
p300 = protocol.load_instrument('p300_single_gen2', 'left', tip_racks=[tips])
#p20 = protocol.load_instrument('p20_single_gen2', 'right', tip_racks=[tips])

def main():

    #test-defined values
    #Adjust these values based on the lateral flow assay protocol
    
    #INGEZIM ASF CROM Ag
    num_wells = 1
    duo_config = False
    uses_buffer = True
    immediately_add_buffer = False
    leftmost_well_for_sample = True
    buffer_in_which_well = 1
    
    num_drops_sample = 1
    sample_vol = 50 #ul
    sample_vol_per_drop = 50

    num_drops_buffer = 1
    buffer_vol = 200
    buffer_vol_per_drop = 200
    
    #DRY RUN: pick up tip
    p300.pick_up_tip(tips.wells()[0])
    
    #transfer sample liquid to cassettes
    transfer_samples(sample_vol)
    
    #transfer X drops of running buffer to cassettes
    if(uses_buffer == True):
        for i in range(num_drops_buffer):
            transfer_buffer(buffer_vol_per_drop)
    
    #DRY RUN: drop tip
    p300.drop_tip(tips.wells()[0])
    
    #rehome
    protocol.home()
    
    #print console output for simulation data
    for line in protocol.commands():
        print(line)

#fin
        
def transfer_samples(sample_vol):
    for i in range(5):

        #determining which slots to dispense on for this iteration of the loop
        if (i == 0 or i == 1):
            start_slot = labware_1
            mid_slot = labware_4
            fin_slot = labware_7
        elif (i == 2):
            start_slot = labware_2
            mid_slot = labware_5
            fin_slot = labware_8
        else:
            start_slot = labware_3
            mid_slot = labware_6
            fin_slot = labware_9

        #loading_wells: list of wells being loaded with sample liquid
        if ((i % 2 == 0 and i != 4) or i == 3):
            loading_wells = ['C1', 'B1', 'A1']
            loading_wells2 = ['C1', 'B1']
        else:
            loading_wells = ['C2', 'B2', 'A2']
            loading_wells2 = ['C2', 'B2']
        
        #transfers
        row = sample_plate.rows()[i]
        p300.transfer(
            sample_vol, 
            row[:3], 
            [start_slot.wells_by_name()[well_name] for well_name in loading_wells],
            new_tip='never')   #trash=False will cause pipette with tip to hover over next tip
        p300.transfer(
            sample_vol, 
            row[3:6], 
            [mid_slot.wells_by_name()[well_name] for well_name in loading_wells],
            new_tip='never')
        p300.transfer(
            sample_vol, 
            row[6:8], 
            [fin_slot.wells_by_name()[well_name] for well_name in loading_wells2],
            new_tip='never')

    p300.transfer(
        sample_vol, 
        [sample_plate.wells_by_name()[well_name] for well_name in ['F1', 'F2']], 
        [labware_7.wells_by_name()[well_name] for well_name in ['A1', 'A2']],
        new_tip='never')

def transfer_buffer(buffer_vol_per_drop):
    for i in range(5):
        
        #determining which slots to dispense on for this iteration of the loop
        if (i == 0 or i == 1):
            start_slot = labware_1
            mid_slot = labware_4
            fin_slot = labware_7
        elif (i == 2):
            start_slot = labware_2
            mid_slot = labware_5
            fin_slot = labware_8
        else:
            start_slot = labware_3
            mid_slot = labware_6
            fin_slot = labware_9

        #loading_wells: list of wells being loaded with running buffer
        if ((i % 2 == 0 and i != 4) or i == 3):
            loading_wells = ['C1', 'B1', 'A1']
            loading_wells2 = ['C1', 'B1']
        else:
            loading_wells = ['C2', 'B2', 'A2']
            loading_wells2 = ['C2', 'B2']

        #listing wells F3-F8 which contain running buffer
        buffer_wells = ['F3', 'F4', 'F5', 'F6', 'F7', 'F8']
        
        #distributes
        #TODO: determine if disposal_volume should be increased for more consistent dispense volumes at the cost of time and buffer
        p300.distribute(
            buffer_vol_per_drop, 
            [sample_plate.wells_by_name()[well_name] for well_name in buffer_wells], 
            [start_slot.wells_by_name()[well_name] for well_name in loading_wells], 
            new_tip='never',
            disposal_volume=0)
        p300.distribute(
            buffer_vol_per_drop, 
            [sample_plate.wells_by_name()[well_name] for well_name in buffer_wells], 
            [mid_slot.wells_by_name()[well_name] for well_name in loading_wells],
            new_tip='never',
            disposal_volume=0)
        p300.distribute(
            buffer_vol_per_drop, 
            [sample_plate.wells_by_name()[well_name] for well_name in buffer_wells], 
            [fin_slot.wells_by_name()[well_name] for well_name in loading_wells2],
            new_tip='never',
            disposal_volume=0)

    p300.distribute(
        buffer_vol_per_drop, 
        [sample_plate.wells_by_name()[well_name] for well_name in buffer_wells], 
        [labware_7.wells_by_name()[well_name] for well_name in ['A1', 'A2']],
        new_tip='never',
        disposal_volume=0)
    
if __name__ == "__main__":
    main()

/data/robot_settings.json not found. Loading defaults
Failed to initialize character device, will not be able to control gpios (lights, button, smoothiekill, smoothie reset). Only one connection can be made to the gpios at a time. If you need to control gpios, first stop the robot server with systemctl stop opentrons-robot-server. Until you restart the server with systemctl start opentrons-robot-server, you will be unable to control the robot using the Opentrons app.
/data/deck_calibration.json not found. Loading defaults


KeyboardInterrupt: 

In [2]:
#run if you stop dry run protocol halfway through and need to return tip

#returns pipette tip and homes
import opentrons.execute
protocol = opentrons.execute.get_protocol_api('2.13')
protocol.home()

#assigns each OT2 slot a custom labware definition that corresponds to the cutout

labware_1 = protocol.load_labware("rtwizard_slot_147_rapid_asfv_ag", location="1")
labware_1.set_offset(x=-6.50, y=-4.30, z=-5.00)

labware_2 = protocol.load_labware("rtwizard_slot_258_rapid_asfv_ag", location="2")
labware_2.set_offset(x=-1.20, y=-4.40, z=-5.00)

labware_3 = protocol.load_labware("rtwizard_slot_369_rapid_asfv_ag", location="3")
labware_3.set_offset(x=4.00, y=-4.00, z=-5.00)

labware_4 = protocol.load_labware("rtwizard_slot_147_rapid_asfv_ag", location="4")
labware_4.set_offset(x=-6.50, y=-2.00, z=-4.60)

labware_5 = protocol.load_labware("rtwizard_slot_258_rapid_asfv_ag", location="5")
labware_5.set_offset(x=-1.00, y=-2.00, z=-4.00)

labware_6 = protocol.load_labware("rtwizard_slot_369_rapid_asfv_ag", location="6")
labware_6.set_offset(x=3.80, y=-2.00, z=-4.00)

labware_7 = protocol.load_labware("rtwizard_slot_147_rapid_asfv_ag", location="7")
labware_7.set_offset(x=-6.10, y=-0.30, z=-4.00)

labware_8 = protocol.load_labware("rtwizard_slot_258_rapid_asfv_ag", location="8")
labware_8.set_offset(x=-0.40, y=0.50, z=-4.00)

labware_9 = protocol.load_labware("rtwizard_slot_369_rapid_asfv_ag", location="9")
labware_9.set_offset(x=4.90, y=0.20, z=-4.00)

labware_10 = protocol.load_labware("rtwizard_48_wellplate_sarstedt_1100ul_tubes", location="10")
labware_10.set_offset(x=-1.00, y=2.00, z=-3.00)

labware_11 = protocol.load_labware("opentrons_96_tiprack_300ul", location="11")
labware_11.set_offset(x=-0.60, y=7.40, z=0.00)

tips = labware_11
#comment or uncomment depending on pipette setup
#comment or uncomment depending on pipette setup
p300 = protocol.load_instrument('p300_single_gen2', 'left', tip_racks=[tips])
#p20 = protocol.load_instrument('p20_single_gen2', 'right', tip_racks=[tips])

#DRY RUN: drop tip back in tiprack
p300.drop_tip(tips.wells()[0])

#DRY RUN: drop broken tip in trash
#small_pipette.drop_tip()

#rehome
protocol.home()
    
#print console output
for line in protocol.commands():
    print(line)



Dropping tip into A1 of Opentrons OT-2 96 Tip Rack 20 µL on 11
